# Простейшая реализация линейной регрессии

In [1]:
import numpy as np

In [2]:
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Загрузим данные из sklearn (стандартный датасет для регрессии, в котором надо предсказать стоимость дома по его характеристикам):

In [3]:
boston = load_boston()
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR'])

In [4]:
X = boston["data"]
y = boston["target"]

In [5]:
X

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [6]:
X.shape

(506, 13)

Добавьте единичный столбец в X и сохраните результат в переменную X_with1:

In [9]:
### your code here
# (506, 13) -> (506, 14)
ones = np.ones(X.shape[0]) # 506
X_with1 = np.hstack((X, ones[:, np.newaxis]))

In [10]:
X_with1

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 3.9690e+02, 4.9800e+00,
        1.0000e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 3.9690e+02, 9.1400e+00,
        1.0000e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 3.9283e+02, 4.0300e+00,
        1.0000e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 5.6400e+00,
        1.0000e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 3.9345e+02, 6.4800e+00,
        1.0000e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 7.8800e+00,
        1.0000e+00]])

Разделим обе выборки на обучение и контроль, зафиксировав random_state, чтобы разбиение в обоих случаях выполнялось одинаково:

In [11]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, random_state=10)
X_tr_with1, X_te_with1, y_tr, y_te = train_test_split(X_with1, y, random_state=10)

In [12]:
X_tr.shape, y_tr.shape, X_tr_with1.shape, X_te_with1.shape

((379, 13), (379,), (379, 14), (127, 14))

Реализуйте методы fit и predict (в методе fit нужно посчитать веса и сохранить в self., в методе predict - вернуть предсказания, используя сохраненный вектор весов). Интерфейс соответствует стандартному интерфейсу sklearn. Реализовывать поддержку learning_method="gd" пока не надо:

In [13]:
class LinearRegression:
    def __init__(self, learning_method="analytical"):
        self.learning_method = learning_method
        
    def fit(self, X, y):
        if self.learning_method == "analytical":
            ### your code here
            dxd = np.linalg.inv(np.dot(X.T, X))
            dx1 = np.dot(X.T, y)
            self.w = np.dot(dxd, dx1)
        else:
            pass
            
    def predict(self, X):
        ### your code here
        return np.dot(X, self.w)

Проверим качество работы своего метода регрессии. Сравним работу метода без константного признака и с ним:

In [14]:
regr = LinearRegression("analytical")
regr.fit(X_tr, y_tr)
print("train error:", mean_squared_error(regr.predict(X_tr), y_tr))
print("test error:", mean_squared_error(regr.predict(X_te), y_te))

train error: 20.700912370553027
test error: 35.911785868585646


In [15]:
regr = LinearRegression("analytical")
regr.fit(X_tr_with1, y_tr)
print("train error:", mean_squared_error(regr.predict(X_tr_with1), y_tr))
print("test error:", mean_squared_error(regr.predict(X_te_with1), y_te))

train error: 18.877581397105875
test error: 32.46387291045164


Вы также можете реализовать обучение линейной регрессии градиентным спуском. В этом случае может понадобиться учесть несколько тонкостей:
* Выборку X необходимо нормировать с помощью sklearn.preprocessing.Normalizer
* Длину шга нужно настроить так, чтобы градиентный спуск не расходился (начните с большой, постепенно уменьшайте, пока ошибка на обучении не станет нормального масштаба)
* Может понадобиться сделать много итераций (порядка миллиона)